In [7]:
import os
import socket
import subprocess
import pandas as pd
import io
from contextlib import redirect_stdout
import itertools
import warnings
from datetime import datetime
import time
import timeout_decorator
import numpy as np
import matplotlib.pyplot as plt



In [8]:
def islocal(): return socket.gethostname()=="rc-idsia-macBook.local"
def isserver(): return socket.gethostname()=="ec6120930b2d"
def strdate(): return datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
def ishomeserver(): return socket.gethostname()=='rc-aau-macBook'

if islocal():
    prj_path = "/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA"
    os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home"
elif ishomeserver():
    prj_path = "/Users/rcabanas/work_java_crema/crema/CreMA"
    os.environ['JAVA_HOME']="/Library/Java/JavaVirtualMachines/adoptopenjdk-12.jdk/Contents/Home"
elif isserver():
    prj_path="/home/rcabanas/work_java_crema/crema/CreMA"
else:
    warnings.warn("Unknown server, set prj_path variable manually.")
os.chdir(prj_path)
!pwd

/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA


In [9]:
exp_folder = f"{prj_path}/papers/pgm20/experiments/"
log_folder = f"{exp_folder}/logs/"
res_folder = f"{exp_folder}/results/"
jarfile=f"{prj_path}/target/CreMA-0.1.1-SNAPSHOT-jar-with-dependencies.jar"

rebuild = True

In [10]:
if rebuild:
    !mvn install:install-file -Dfile=./external/lpsolve55j.jar -DgroupId=net.sf.lpsolve -DartifactId=lp_solve -Dversion=5.5.2 -Dpackaging=jar
    !mvn install:install-file -Dfile=./external/polco.jar -DgroupId=ch.javasoft.polco -DartifactId=polco -Dversion=4.7.1 -Dpackaging=jar
    !mvn clean compile assembly:single

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for ch.idsia:CreMA:jar:0.1.1-SNAPSHOT
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.apache.commons:commons-math3:jar -> duplicate declaration of version 3.6.1 @ line 125, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ---------------------------< ch.idsia:CreMA >---------------------------
[INFO] Building Credal Model Algorithms 0.1.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-install-plugin:2.4:install-file (default-cli) @ CreMA ---
[INFO] Installing /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA/external/lpsolve55j.j

In [12]:
disable_java = False
@timeout_decorator.timeout(7*1*60)
def run_java(jarfile, javafile):
    cmd=f"java -cp {jarfile} {javafile}"
    if disable_java: cmd= "echo 0,0,0,0,0,0"    
    print(cmd)
    result =subprocess.run(cmd, universal_newlines = True, shell=True,stdout = subprocess.PIPE)
    print(result.stdout)
    return result

# ChainNonMarkovian 6 5 1 -1 0 CCALP 1234
def run_chain(model, N, endovarsize, exovarsize, target, obsvar, dovar, method, seed):
    
    
    print(strdate())
    exovarsize = exovarsize or endovarsize*endovarsize + 1
    
    if obsvar is None: obsvar = -1;
    elif obsvar<0: obsvar = N + obsvar;
    
    if dovar is None: dovar = -1;    
    elif dovar<0: dovar = N + dovar;
    
    if target is None: target = N//2;
    elif target<0: target = N + target;
    
    warmups = 0
    repetitions = 1
    eps = 0.0001
    #ChainNonMarkovian 4  -v 5 -V 9 -t 1 -o -1 -d 0 -m  CCALP --seed 12234 --warmpus 0 --repetitions 1
    javafile = f"{exp_folder}/RunExperiments.java {model} {N} -v {endovarsize} -V {exovarsize} -t {target} -o {obsvar} -d {dovar} -m {method} -e {eps} -s {seed} -w {warmups} -r {repetitions} "  
    try:
        result = run_java(jarfile, javafile)
        output = [float(x) for x in result.stdout.splitlines()[-1].split(",")]
    except:
        output = [float("inf"),float("inf")]+[float("nan")]* (endovarsize*2)
        
    return output

def run_chain_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("ChainMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_chain_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("ChainNonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)


def run_hmm_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("HMM-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_hmm_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("HMM-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)



def run_rhmm_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("RHMM-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_rhmm_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("RHMM-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_squares_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("Squares-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_squares_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("Squares-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)



def run_chain_terbin_markovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("TerBinChainMarkovian", N, 2, None, target, obsvar, dovar, method, seed)

def run_chain_terbin_nonmarkovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("TerBinChainNonMarkovian", N, 2, None, target, obsvar, dovar, method, seed)



In [13]:
run_hmm_nonmarkovian(N=5, method="CCVE")

2020-05-26-09-42-03
java -cp /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA/target/CreMA-0.1.1-SNAPSHOT-jar-with-dependencies.jar /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA/papers/pgm20/experiments//RunExperiments.java HMM-NonMarkovian 5 -v 2 -V 5 -t 2 -o 4 -d 0 -m CCVE -e 0.0001 -s 1234 -w 0 -r 1 

HMM-NonMarkovian
   N=5 endovarsize=2 exovarsize=5 target=2 obsvar=4 dovar=0 method=CCVE seed=1234
Running experiments...
deleted = [1, 3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17]
Measurement #0 in 872.659 ms.
[0.9660574412532636, 0.9660574412532636]
872.659,44.734,0.9660574412532636,0.9660574412532636,0.033942558746736295,0.033942558746736295



[872.659,
 44.734,
 0.9660574412532636,
 0.9660574412532636,
 0.033942558746736295,
 0.033942558746736295]

In [14]:

def run_experiments(f, args, outkeys, fargs=None, verbose=False, lenght_dep_vars = None, non_evaluable=[]):
    
    
    
    print("=========")
    print(args)
    print("=========")         
        
    result = pd.DataFrame(columns=list(args.keys())+list(outkeys))
    log_file = f"{log_folder}{strdate()}_{f.__name__}.txt"
    
    data = pd.DataFrame(list(itertools.product(*list(args.values()))), columns = args.keys())
    
    fargs = fargs or {}
    for k,v in fargs.items():
        data[k]=data.apply(v, axis=1)
        
        
    non_evaluable = non_evaluable or [] 
    
    lenght_dep_vars = lenght_dep_vars or ["N"]
    
    def is_evaluable(args):
        current = {k:v for (k,v) in args.items() if k not in lenght_dep_vars}
        previous = [{k:v for (k,v) in a.items() if k not in lenght_dep_vars} for a in non_evaluable]
        
        print(f"current: {current}")
        print(f"previous: {previous}")

        
        return not current in previous
       
    def single_experiment(argsv):
        if is_evaluable(argsv):
            outvals = f(**argsv)
            if np.isnan(outvals).any(): 
                non_evaluable.append(argsv)
                print(f"setting as not evaluable: {argsv}")
        else:
            outvals=[float("nan")]*len(outkeys)
        return outvals    
        
    
    for argsv in data.to_dict(orient="row"):    
        
        strio = io.StringIO()
        
           
        
        with open(log_file, 'a+') as logger:
            if verbose == False:
                with redirect_stdout(strio):
                    outvals = single_experiment(argsv)
            else:
                outvals = single_experiment(argsv)   
            
            logger.write(strio.getvalue())
            
            
        result = result.append({**argsv, **dict(zip(list(outkeys), outvals))}, ignore_index=True)
        print(dict(result.iloc[-1]))
        print("\n\n")
    
    return result

def get_args(**kwargs): return kwargs

##




In [15]:
SEED = [1234+i*1234//2 for i in range(0,100,2)]
res = {}
#SEED = [1234+i*1234 for i in range(3)]
len(SEED)

50

In [ ]:
## with evidence chain NON markovian ... RUN
endovarsize = 2    
args = dict(N=range(3,11), dovar=[0], obsvar=[-1], seed=SEED, exovarsize=[6], method=["CCVE", "CCALPeps"])
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])


results1 = run_experiments(run_chain_nonmarkovian, args, outkeys)
res_file = f"{res_folder}{strdate()}_chain_nonmarkovian_prec.csv"
results1.to_csv(res_file)
                     
res["chain_nonmarkovian_prec"] = results1

In [ ]:
# with evidence HMM nonmarkovian 

endovarsize = 2
args = dict(N=range(3,11), dovar=[0], seed=SEED, exovarsize=[6], method=["CCVE","CCALPeps"])
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
             obsvar = lambda t: int(t["N"]*2 - 1))

lenght_dep_vars = ["N", "target", "obsvar"]

results1 = run_experiments(run_hmm_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
res_file = f"{res_folder}{strdate()}_hmm_nonmarkovian_prec.csv"
results1.to_csv(res_file)

res["hmm_nonmarkovian_prec"] = results1

In [16]:
# with evidence RevHMM nonmarkovian ... RUN
endovarsize=2
args = dict(N=range(3,11), dovar=[0], seed=SEED, exovarsize=[6], method=["CCVE","CCALPeps"])
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
             obsvar = lambda t: int(t["N"]*2 - 2))

lenght_dep_vars = ["N", "target", "obsvar"]

results1 = run_experiments(run_rhmm_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
res_file = f"{res_folder}{strdate()}_rhmm_nonmarkovian_prec.csv"
results1.to_csv(res_file)

res["rhmm_nonmarkovian_prec"] = results1

{'N': range(3, 11), 'dovar': [0], 'seed': [1234, 2468, 3702, 4936, 6170, 7404, 8638, 9872, 11106, 12340, 13574, 14808, 16042, 17276, 18510, 19744, 20978, 22212, 23446, 24680, 25914, 27148, 28382, 29616, 30850, 32084, 33318, 34552, 35786, 37020, 38254, 39488, 40722, 41956, 43190, 44424, 45658, 46892, 48126, 49360, 50594, 51828, 53062, 54296, 55530, 56764, 57998, 59232, 60466, 61700], 'exovarsize': [6], 'method': ['CCVE', 'CCALPeps']}
{'N': 3, 'dovar': 0, 'seed': 1234, 'exovarsize': 6, 'method': 'CCVE', 'time': 763.218, 'query_time': 138.008, 'lowerbound0': 0.45746409650015196, 'upperbound0': 0.45746409650015196, 'lowerbound1': 0.542535903499848, 'upperbound1': 0.542535903499848, 'obsvar': 4.0, 'target': 2.0}



{'N': 3, 'dovar': 0, 'seed': 1234, 'exovarsize': 6, 'method': 'CCALPeps', 'time': 1752.3, 'query_time': 1678.577, 'lowerbound0': 0.4574312376913952, 'upperbound0': 0.45745920643605514, 'lowerbound1': 0.5425407935639448, 'upperbound1': 0.5425687623086048, 'obsvar': 4.0, 'target': 

In [10]:
# with evidence Squares nonmarkovian 

endovarsize = 2
args = dict(N=list(range(3,11)), dovar=[0], seed=SEED, exovarsize=[6], method=["CCVE","CCALPeps"])
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
             obsvar = lambda t: int(t["N"]*2 - 1))

lenght_dep_vars = ["N", "target", "obsvar"]

results1 = run_experiments(run_squares_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
res_file = f"{res_folder}{strdate()}_squares_nonmarkovian_prec.csv"
results1.to_csv(res_file)

res["squares_nonmarkovian_prec"] = results1


{'N': [3, 4, 5, 6, 7, 8, 9, 10], 'dovar': [0], 'seed': [1234, 2468, 3702, 4936, 6170, 7404, 8638, 9872, 11106, 12340, 13574, 14808, 16042, 17276, 18510, 19744, 20978, 22212, 23446, 24680, 25914, 27148, 28382, 29616, 30850, 32084, 33318, 34552, 35786, 37020, 38254, 39488, 40722, 41956, 43190, 44424, 45658, 46892, 48126, 49360, 50594, 51828, 53062, 54296, 55530, 56764, 57998, 59232, 60466, 61700], 'exovarsize': [6], 'method': ['CCVE', 'CCALPeps']}
{'N': 3, 'dovar': 0, 'seed': 1234, 'exovarsize': 6, 'method': 'CCVE', 'time': 2300.97, 'query_time': 542.233, 'lowerbound0': 0.5162224745420247, 'upperbound0': 0.5162224745420247, 'lowerbound1': 0.48377752545797525, 'upperbound1': 0.48377752545797525, 'obsvar': 5.0, 'target': 2.0}



{'N': 3, 'dovar': 0, 'seed': 1234, 'exovarsize': 6, 'method': 'CCALPeps', 'time': 4008.819, 'query_time': 3865.333, 'lowerbound0': 0.5160364962038373, 'upperbound0': 0.5160638274485072, 'lowerbound1': 0.48393617255149285, 'upperbound1': 0.4839635037961627, 'obsvar'

In [ ]:
def foo(*args, **kwargs):
    print(args)
    print(kwargs)
run_experiments(foo, args, outkeys)